In [ ]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
dfAuthor = pd.DataFrame()
display(dfAuthor.head())

In [ ]:
#Search DBPedia for Authors
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
query = """
    SELECT DISTINCT ?s ?label ?birthDate ?deathDate
      WHERE
        {
          ?s a dbo:Writer .
          ?s dbo:birthDate ?birthDate .
          ?s dbo:deathDate ?deathDate .
          ?s rdfs:label ?label .
        }
        LIMIT 10000 OFFSET 90000
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

#display(results)


In [ ]:
#Add Meta Data - Run only once.
result = results["results"]["bindings"][0]
display(result)
#Add meta data
author={}
author['URI'] = 'URI'
author['label'] = 'http://www.w3.org/2000/01/rdf-schema#label'
author['birthDate'] = 'http://dbpedia.org/ontology/birthDate'
author['deathDate'] = 'http://dbpedia.org/ontology/deathDate'
dfNewAuthor = pd.DataFrame(author, index=[0])
dfAuthor = dfAuthor.append(dfNewAuthor, ignore_index=True, sort=True)

#Add meta data - datatypes
author={}
author['URI'] = 'URI'
author['label'] = 'XMLSchema#string'
author['birthDate'] = result['birthDate']['datatype'].split('/')[-1]
author['deathDate'] = result['deathDate']['datatype'].split('/')[-1]
dfNewAuthor = pd.DataFrame(author, index=[0])
dfAuthor = dfAuthor.append(dfNewAuthor, ignore_index=True, sort=True)

#Add meta data - datatypes
author={}
author['URI'] = 'http://www.w3.org/2002/07/owl#Thing'
author['label'] = 'http://www.w3.org/2001/XMLSchema#string'
author['birthDate'] = result['birthDate']['datatype']
author['deathDate'] = result['deathDate']['datatype']
dfNewAuthor = pd.DataFrame(author, index=[0])
dfAuthor = dfAuthor.append(dfNewAuthor, ignore_index=True, sort=True)
#dfAuthor = dfAuthor.append(dfNewAuthor, ignore_index=True, sort=True)

cols = dfAuthor.columns.tolist()
cols = cols[-1:] + cols[:-1]

dfAuthor = dfAuthor[cols] 
display(dfAuthor)

In [ ]:
for result in results["results"]["bindings"]:
    author = {}
    if result['s']['value'] not in dfAuthor['URI'].values:
        if result['label']['xml:lang'] == 'en':
            author['URI'] = result['s']['value']
            author['label'] = result['label']['value']
            author['birthDate'] = result['birthDate']['value']
            author['deathDate'] = result['deathDate']['value']
            dfNewAuthor = pd.DataFrame(author, index=[0])
            dfAuthor = dfAuthor.append(dfNewAuthor, ignore_index=True, sort=True)

display(dfAuthor)

In [ ]:
dfAuthorWithWork = pd.DataFrame()
for index, row in dfAuthor.iterrows():
    if index == 0:
        row['work'] = 'http://dbpedia.org/ontology/work'
        dfAuthorWithWork = dfAuthorWithWork.append(row)
    if index == 1:
        row['work'] = 'XMLSchema#list'
        dfAuthorWithWork = dfAuthorWithWork.append(row)
    if index == 2:
        row['work'] = 'https://www.w3.org/2001/XMLSchema#list'
        dfAuthorWithWork = dfAuthorWithWork.append(row)
    if index > 2:
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        query = """SELECT DISTINCT * WHERE { ?s a dbo:Work . ?s dbo:author <%s>  . } LIMIT 10000""" % row['URI']
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        work = ''
        if len(results["results"]["bindings"]) > 0:
            for result in results["results"]["bindings"]:
                work = work + '\t' + result['s']['value'].replace('http://dbpedia.org/resource/', '')
            row['work'] = work
            dfAuthorWithWork = dfAuthorWithWork.append(row)

display(dfAuthorWithWork)

In [ ]:
dfAuthorWithSameAs = pd.DataFrame()
for index, row in dfAuthor.iterrows():
    if index == 0:
        row['work'] = 'http://dbpedia.org/ontology/work'
        dfAuthorWithWork = dfAuthorWithWork.append(row)
    if index == 1:
        row['work'] = 'XMLSchema#list'
        dfAuthorWithWork = dfAuthorWithWork.append(row)
    if index == 2:
        row['work'] = 'https://www.w3.org/2001/XMLSchema#list'
        dfAuthorWithWork = dfAuthorWithWork.append(row)
    if index > 2:
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        query = """SELECT DISTINCT * WHERE { ?s a dbo:Work . ?s dbo:author <%s>  . } LIMIT 10000""" % row['URI']
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        work = ''
        if len(results["results"]["bindings"]) > 0:
            for result in results["results"]["bindings"]:
                work = work + '\t' + result['s']['value'].replace('http://dbpedia.org/resource/', '')
            row['work'] = work
            dfAuthorWithWork = dfAuthorWithWork.append(row)

display(dfAuthorWithWork)